# Entity Resolution on Unlabeled Text Data

#### By Ahsan Khan

In [2]:
# Set up Library imports

import pandas as pd
import re
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
#from sklearn.metrics import jaccard_similarity_score

In [3]:
# Read csv files into Pandas dataframes

entities_df = pd.read_csv('All entities.csv')

targets_df = pd.read_csv('All targets.csv')

# deep copy
intact_entities_df = entities_df.copy()
intact_targets_df = targets_df.copy()

In [4]:
entities_df

,All entities
0,BLACKROCK ADV UKLTD-MORGAN-AGG
1,BLACKROCK FIN MG AAF-MORGANTRN
2,Commingled Pension Trust Fund (Core Bond) of J...
3,COMMINGLED PENSION TRUST FUND (CORE PLUS BOND)...
4,COMMINGLED PENSION TRUST FUND (CORE PLUS BOND)...
...,...
367,MORGAN STANLEY & CO INTERNATIONAL PLC
368,MORGAN STANLEY & CO INTL P-GBR
369,MORGAN STANLEY & CO INTL P-GBR
370,MORGAN STANLEY & CO INTL P-IOS


In [5]:
targets_df

,Target
0,Blackrock Advisors (UK) Limited
1,"BlackRock Financial Management, Inc."
2,Commingled Pension Trust Fund
3,Commingled Pension Trust Fund
4,Commingled Pension Trust Fund
...,...
367,Morgan Stanley & Co. International plc
368,Morgan Stanley & Co. International plc
369,Morgan Stanley & Co. International plc
370,Morgan Stanley & Co. International plc


### Preprocessing 

In the preprocessing state we are removing all punctuation, numbers and converting all text to lowercase. This will ensure uniformity in the corpus and all incoming entities

In [6]:
# Remove punctuation and numbers, and convert to lower case 

def preprocess(df):
    df.iloc[:,0] = df.iloc[:,0].apply(lambda x : str.lower(x)) # lower case
    df.iloc[:,0] = df.iloc[:,0].apply\
            (lambda x : " ".join(re.findall('[\w]+',x))) # remove punc, nums 
    return df

In [7]:
entities_df.iloc[:,0]

0                         BLACKROCK ADV UKLTD-MORGAN-AGG
1                         BLACKROCK FIN MG AAF-MORGANTRN
2      Commingled Pension Trust Fund (Core Bond) of J...
3      COMMINGLED PENSION TRUST FUND (CORE PLUS BOND)...
4      COMMINGLED PENSION TRUST FUND (CORE PLUS BOND)...
                             ...                        
367                MORGAN STANLEY & CO INTERNATIONAL PLC
368                       MORGAN STANLEY & CO INTL P-GBR
369                       MORGAN STANLEY & CO INTL P-GBR
370                       MORGAN STANLEY & CO INTL P-IOS
371                         MORGAN STANLEY & CO IN11-LCW
Name: All entities, Length: 372, dtype: object

### Generating 3 character tokens (markov N-grams)

In [8]:
# A function for generating markov tokens 

remove_spaces = lambda s : "".join(s.split())

def split_3_char(s):
    if len(s) == 3:
        return s
    return s[:3] + ' ' + split_3_char(s[1:])

In [9]:
# Generating markov tokens of our dataframe

def tokenize_all(df):
    l = list(df.iloc[:,0])
    Tokens = []
    for w in l:
        w = remove_spaces(w)
        tok = split_3_char(w)
        Tokens.append(tok)
    return Tokens

### Checking jaccard similarty (scraped)

It was found that Jaccard similarity was not giving us the best match even though it is known best for text data. Say we have the token "ele" 3 repeat times in the title of an entity. Jaccard similarity will count it as just 1 "ele" and hence defeat the purpose. Cosine similarity will give us a better matching.

In [10]:
# jaccard similartiy function

def get_jaccard_sim(str1, str2): 
    a = set(str1.split()) 
    b = set(str2.split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

## Preprocessing

In [11]:
# Running our functions on data

entities_df = preprocess(entities_df)
targets_df = preprocess(targets_df)

In [12]:
entities_df

,All entities
0,blackrock adv ukltd morgan agg
1,blackrock fin mg aaf morgantrn
2,commingled pension trust fund core bond of jpm...
3,commingled pension trust fund core plus bond o...
4,commingled pension trust fund core plus bond o...
...,...
367,morgan stanley co international plc
368,morgan stanley co intl p gbr
369,morgan stanley co intl p gbr
370,morgan stanley co intl p ios


In [13]:
targets_df

,Target
0,blackrock advisors uk limited
1,blackrock financial management inc
2,commingled pension trust fund
3,commingled pension trust fund
4,commingled pension trust fund
...,...
367,morgan stanley co international plc
368,morgan stanley co international plc
369,morgan stanley co international plc
370,morgan stanley co international plc


## Tokenization 

In [14]:
ent_corpus = tokenize_all(entities_df)
targ_corpus = tokenize_all(targets_df)

In [15]:
targ_corpus[3]

'com omm mmi min ing ngl gle led edp dpe pen ens nsi sio ion ont ntr tru rus ust stf tfu fun und'

In [16]:
ent_corpus[3]

'com omm mmi min ing ngl gle led edp dpe pen ens nsi sio ion ont ntr tru rus ust stf tfu fun und ndc dco cor ore rep epl plu lus usb sbo bon ond ndo dof ofj fjp jpm pmo mor org rga gan anc nch cha has ase seb eba ban ank nkn kna'

In [17]:
ent_size = len(ent_corpus)
corpus_ent_targ = ent_corpus + targ_corpus # A giant corpus containing both entities and targets markov tokens.

In [18]:
ent_size

372

In [19]:
# Main corpus

len(corpus_ent_targ)

744

## Cosine similariy


#### Making TF matrix (not tf-idf)

In [20]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
tf = vectorizer.fit_transform(corpus_ent_targ)

After applying tf-idf scores I noticed the 'idf' part was giving less weights to tokens that appeared more often in the corpus. In our case this is not what we want. We want all the tokens to be given equal weights as each token represents a chunk of the entity's title. Hence, just tf scoring and not tf-idf scoring is better in this case.

Now we have a sparse maxtrix of all tokens

In [21]:
print(tf)

  (0, 130)	1
  (0, 461)	1
  (0, 59)	1
  (0, 165)	1
  (0, 457)	1
  (0, 726)	1
  (0, 631)	1
  (0, 162)	1
  (0, 447)	1
  (0, 64)	1
  (0, 239)	1
  (0, 948)	1
  (0, 911)	1
  (0, 454)	1
  (0, 523)	1
  (0, 833)	1
  (0, 220)	1
  (0, 547)	1
  (0, 656)	1
  (0, 712)	1
  (0, 355)	1
  (0, 86)	1
  (0, 554)	1
  (0, 68)	1
  (1, 130)	1
  :	:
  (743, 427)	1
  (743, 121)	1
  (743, 857)	1
  (743, 425)	1
  (743, 612)	1
  (743, 845)	1
  (743, 97)	1
  (743, 679)	1
  (743, 610)	1
  (743, 806)	1
  (743, 826)	1
  (743, 95)	1
  (743, 603)	1
  (743, 487)	1
  (743, 329)	1
  (743, 558)	1
  (743, 643)	1
  (743, 555)	1
  (743, 961)	1
  (743, 300)	1
  (743, 725)	1
  (743, 80)	1
  (743, 172)	1
  (743, 638)	1
  (743, 520)	1


Now finding cosine similarity of each entity with every target.`

In [26]:
# cosine on Tf

similarity_ent_targ = cosine_similarity(tf[0:ent_size], tf[ent_size:])
similarity_ent_targ # List of lists containing each entity and its weights

array([[0.45833333, 0.34114412, 0.        , ..., 0.15161961, 0.15161961,
        0.15161961],
       [0.29166667, 0.37904902, 0.04166667, ..., 0.15161961, 0.15161961,
        0.15161961],
       [0.        , 0.02550722, 0.67292658, ..., 0.12753608, 0.12753608,
        0.12753608],
       ...,
       [0.        , 0.        , 0.        , ..., 0.64835272, 0.64835272,
        0.64835272],
       [0.        , 0.        , 0.        , ..., 0.64835272, 0.64835272,
        0.64835272],
       [0.        , 0.        , 0.        , ..., 0.6228411 , 0.6228411 ,
        0.6228411 ]])

In [43]:
pd.DataFrame(similarity_ent_targ)

,0,1,2,3,4,5,6,7,8,9,...,362,363,364,365,366,367,368,369,370,371
0,0.458333,0.341144,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.246183,0.246183,0.246183,0.204124,0.204124,0.151620,0.151620,0.151620,0.151620,0.151620
1,0.291667,0.379049,0.041667,0.041667,0.041667,0.041667,0.041667,0.041667,0.041667,0.041667,...,0.246183,0.246183,0.246183,0.204124,0.204124,0.151620,0.151620,0.151620,0.151620,0.151620
2,0.000000,0.025507,0.672927,0.672927,0.672927,0.672927,0.672927,0.672927,0.672927,0.672927,...,0.165663,0.165663,0.165663,0.137361,0.137361,0.127536,0.127536,0.127536,0.127536,0.127536
3,0.000000,0.024596,0.648886,0.648886,0.648886,0.648886,0.648886,0.648886,0.648886,0.648886,...,0.159745,0.159745,0.159745,0.132453,0.132453,0.122980,0.122980,0.122980,0.122980,0.122980
4,0.000000,0.024596,0.648886,0.648886,0.648886,0.648886,0.648886,0.648886,0.648886,0.648886,...,0.159745,0.159745,0.159745,0.132453,0.132453,0.122980,0.122980,0.122980,0.122980,0.122980
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
367,0.000000,0.000000,0.037905,0.037905,0.037905,0.037905,0.037905,0.037905,0.037905,0.037905,...,0.615882,0.615882,0.615882,0.603510,0.603510,1.000000,1.000000,1.000000,1.000000,1.000000
368,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.723747,0.723747,0.723747,0.709208,0.709208,0.648353,0.648353,0.648353,0.648353,0.648353
369,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.723747,0.723747,0.723747,0.709208,0.709208,0.648353,0.648353,0.648353,0.648353,0.648353
370,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.723747,0.723747,0.723747,0.709208,0.709208,0.648353,0.648353,0.648353,0.648353,0.648353


In [44]:
# Index of target with max score for each entry.

targ_indexes = similarity_ent_targ.argmax(axis=1)
targ_indexes #Indexes of targets determined by argmax on cosine similarity weights

array([  0,   1,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,
         2,   2,   2,   2,   2,  18,  18,  18,  18,  18,  18,  18,  18,
        18,  18,  18,  18,  18,  31,  32,  32,  32,  35,  35,  18,  18,
        39,  39, 179, 179, 179, 179, 179, 179, 179, 179, 179,  50,  50,
        50,  53,  53,  53,  56,  56,  56,  56,  56,  56,  62,  62,  64,
        64,  64,  64,  64,  64,  70,  71,  71,  71,  71,  71,  71,  71,
        71,  18,  18,  18,  18,  18,  18,  18,  18,  18,  18,  18,  18,
        91,  31,  31,  92,  92,  92,  92,  98,  31,  31,  31,  31,  31,
        31,  31,  31,  31,  31,  31,  31,  31,  31, 113, 114, 114, 114,
       114, 256, 119, 119, 119, 119, 119, 119, 256, 256, 256, 256, 256,
       256, 256, 256, 133, 133,  50,  50, 137, 137, 137, 140, 140, 140,
        50,  31,  31,  31,  31,  31,  31,  31,  31,  31,  31,  31,  31,
        31,  31,  31,  32,  32,  32,  32,  32,  32,  32,  32,  32,  32,
        32,  32,  31, 172, 173,  18,  18,  18,  18,  18, 179, 17

In [48]:
entities_df = intact_entities_df
targets_df = intact_targets_df

In [49]:
entities_series = entities_df.iloc[:,0]

In [50]:
entities_series

0                         BLACKROCK ADV UKLTD-MORGAN-AGG
1                         BLACKROCK FIN MG AAF-MORGANTRN
2      Commingled Pension Trust Fund (Core Bond) of J...
3      COMMINGLED PENSION TRUST FUND (CORE PLUS BOND)...
4      COMMINGLED PENSION TRUST FUND (CORE PLUS BOND)...
                             ...                        
367                MORGAN STANLEY & CO INTERNATIONAL PLC
368                       MORGAN STANLEY & CO INTL P-GBR
369                       MORGAN STANLEY & CO INTL P-GBR
370                       MORGAN STANLEY & CO INTL P-IOS
371                         MORGAN STANLEY & CO IN11-LCW
Name: All entities, Length: 372, dtype: object

In [51]:
# Get targets given indexes

associated_targets = targets_df.iloc[targ_indexes] 
associated_targets_series = associated_targets.iloc[:,0]
associated_targets_series = associated_targets_series.reset_index(drop=False)

In [52]:
# Concatenating final dataframe

df = pd.concat([entities_series, associated_targets_series], axis=1)

Creating a csv file

In [54]:
df.to_csv('C:\\Users\\Ahsan\\Desktop\\GitHub\\natural_language_processing\\entity_resolution\\matched.csv', index=False)

## Output table of matched entities to its target

In [55]:
df

,All entities,index,Target
0,BLACKROCK ADV UKLTD-MORGAN-AGG,0,Blackrock Advisors (UK) Limited
1,BLACKROCK FIN MG AAF-MORGANTRN,1,"BlackRock Financial Management, Inc."
2,Commingled Pension Trust Fund (Core Bond) of J...,2,Commingled Pension Trust Fund
3,COMMINGLED PENSION TRUST FUND (CORE PLUS BOND)...,2,Commingled Pension Trust Fund
4,COMMINGLED PENSION TRUST FUND (CORE PLUS BOND)...,2,Commingled Pension Trust Fund
...,...,...,...
367,MORGAN STANLEY & CO INTERNATIONAL PLC,270,Morgan Stanley & Co. International plc
368,MORGAN STANLEY & CO INTL P-GBR,256,Morgan Stanley
369,MORGAN STANLEY & CO INTL P-GBR,256,Morgan Stanley
370,MORGAN STANLEY & CO INTL P-IOS,256,Morgan Stanley
